# Lab 4: TableGPT

In this lab, we'll discover the power of code generation models through TableGPT2. The aim is to see how the model can be used in data analysis.

First of all, the notebook is divided into X sections: 0. Installation: This section is dedicated to module installation, model loading and data loading.

1. Guided introduction: Together, we'll discover how to use and evaluate TableGPT2.
2. More questions: You'll need to add at least one new question type to our simple evaluation system.
3. More data sets: You'll need to implement a question with multiple datasets.

IMPORTANT:

- You must work in pairs. You must submit **ONLY ONE NOTEBOOK** for each pair.
- Do not share your work with other pairs.
- You should not use Copilot, ChatGPT or similar tools. At the very least, remove the prompt ...
- <font color='red'>All the things you need to do are indicated in red.</font>

<font color='red'>**FIRST QUESTION:** What are the specificty of the TableGPT2 model?</font> https://huggingface.co/tablegpt/TableGPT2-7B


## 0. Setup


In [61]:
# !pip install transformers datasets bitsandbytes accelerate

In [62]:
from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig,
)

import pandas as pd
import torch

In [63]:
llm_name = "tablegpt/TableGPT2-7B"

# We want to use 4bit quantization to save memory
quantization_config = BitsAndBytesConfig(load_in_8bit=False, load_in_4bit=True)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(llm_name, padding_side="left")
# Prevent some transformers specific issues.
tokenizer.use_default_system_prompt = False
tokenizer.pad_token_id = tokenizer.eos_token_id

# Load LLM.
llm = AutoModelForCausalLM.from_pretrained(
    llm_name,
    quantization_config=quantization_config,
    device_map="cuda",  # load all the model layers on GPU 0
    torch_dtype=torch.bfloat16,  # float precision
)
# Set LLM on eval mode.
llm.eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((3584,), 

In [64]:
generation_config = GenerationConfig(
    max_new_tokens=512,
    do_sample=False,
    # do_sample=True,
    # temperature=.7,
    # top_p=.8,
    # top_k=20,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

In [65]:
df = pd.read_csv("hf://datasets/phihung/titanic/train.csv")
df = df.drop("Cabin", axis=1).dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Name         712 non-null    object 
 4   Sex          712 non-null    object 
 5   Age          712 non-null    float64
 6   SibSp        712 non-null    int64  
 7   Parch        712 non-null    int64  
 8   Ticket       712 non-null    object 
 9   Fare         712 non-null    float64
 10  Embarked     712 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 66.8+ KB


## 1.1 Guided Introduction: The Model.

Below there is an example of a prompt that could be used with TableGPT2.

```
Given access to several pandas dataframes, write the Python code to answer the user's question.
The answer should be store in a variable named "output".

/*
"df.head(5).to_string(index=False)" as follows:
 PassengerId  Survived  Pclass                                                Name    Sex  Age  SibSp  Parch           Ticket    Fare Embarked
           1         0       3                             Braund, Mr. Owen Harris   male 22.0      1      0        A/5 21171  7.2500        S
           2         1       1 Cumings, Mrs. John Bradley (Florence Briggs Thayer) female 38.0      1      0         PC 17599 71.2833        C
           3         1       3                              Heikkinen, Miss. Laina female 26.0      0      0 STON/O2. 3101282  7.9250        S
           4         1       1        Futrelle, Mrs. Jacques Heath (Lily May Peel) female 35.0      1      0           113803 53.1000        S
           5         0       3                            Allen, Mr. William Henry   male 35.0      0      0           373450  8.0500        S
*/

Question: How many child survive? (under 18)
```

The prompt is divided in 3 parts:

1. The global instruction wich is to write python that could answer a question on a specific dataset.
2. The header of the given dataset: 5 first lines of titanic dataset.
3. The question to answer: "How many child survive? (under 18)

First, we will implement a function that generate an answer for this prompt.

<font color='red'>TODO: Fill in the `generate_answer` function following the comments inside.</font>


In [66]:
example_prompt_template = """Given access to several pandas dataframes, write the Python code to answer the user's question.
The answer should be store in a variable named "output".

/*
"{var_name}.head(5).to_string(index=False)" as follows:
{df_info}
*/

Question: {user_question}
"""


def generate_answer(prompt, llm=llm, generation_config=generation_config):
    # Create turns with the given prompt.
    chat = [
        {"role": "user", "content": prompt},
    ]

    # Apply template with the tokenizer. Be careful to return pt tensors on the same device than `llm`.
    chat_encoded = tokenizer.apply_chat_template(chat, return_tensors="pt").to(
        llm.device
    )

    # Generate with llm using the given generation config.
    llm_outputs_ids = llm.generate(
        input_ids=chat_encoded,
        generation_config=generation_config,
    )[0]

    # Decode and select the answer to return.
    answer = tokenizer.decode(
        llm_outputs_ids[chat_encoded.size(1) :], skip_special_tokens=True
    )
    return answer


prompt = example_prompt_template.format(
    var_name="df",
    df_info=df.head(5).to_string(index=False),
    user_question="How many child survive? (under 18)",
)

answer = generate_answer(prompt)

print(prompt)
print("\n*****\n")
print(answer)

Given access to several pandas dataframes, write the Python code to answer the user's question.
The answer should be store in a variable named "output".

/*
"df.head(5).to_string(index=False)" as follows:
 PassengerId  Survived  Pclass                                                Name    Sex  Age  SibSp  Parch           Ticket    Fare Embarked
           1         0       3                             Braund, Mr. Owen Harris   male 22.0      1      0        A/5 21171  7.2500        S
           2         1       1 Cumings, Mrs. John Bradley (Florence Briggs Thayer) female 38.0      1      0         PC 17599 71.2833        C
           3         1       3                              Heikkinen, Miss. Laina female 26.0      0      0 STON/O2. 3101282  7.9250        S
           4         1       1        Futrelle, Mrs. Jacques Heath (Lily May Peel) female 35.0      1      0           113803 53.1000        S
           5         0       3                            Allen, Mr. William Hen

## 1.2 Guided Introduction: The Answer.

As you can see, the model answer with some generated code.

````
Python code:
```python
# Filter the dataframe to include only passengers under the age of 18
children = df[df['Age'] < 18]

# Count the number of children who survived
child_survivors = children[children['Survived'] == 1]

# Save the answer in the variable output
output = len(child_survivors)
````

So we will need to execute it, but there is some difficulty:

1. Sometime, the llm answer with \`\`\`python ... \`\`\`, sometime the llm answer directly with the code. We need to handle both cases.
2. We need to recover the variable output from the execution.
3. We need to evaluate single value and list of values.

First, we will implement a function that generate an answer for this prompt.

<font color='red'>TODO: Fill in the `exec_answer` function following the comments inside.</font>


In [84]:
import re, sys, os


def exec_answer(answer, gold):
    # Extract the code from the answer. Be careful, the code is now always in ``` ```.
    if "```" in answer:
        code = re.search(r"```python\n(.*?)\n```", answer, re.DOTALL)
        code = code.group(1)
    else:
        code = answer

    # Execute the code, https://docs.python.org/3/library/functions.html#exec
    # if the code work: Return True or False based on output == gold (be careful to handle iterable !)
    # if the code don't work return False.

    original_stdout = sys.stdout  # suppress output from exec
    try:
        sys.stdout = open(os.devnull, "w")
        context = {"df": df}
        exec(code, context)
        output = next(reversed(context.values()))

        if isinstance(output, pd.DataFrame) or isinstance(output, pd.Series):
            raise Exception("Output is a DataFrame or Series, please return a scalar.")

        sys.stdout = original_stdout
        return output == gold
    except Exception as e:
        sys.stdout = original_stdout
        print(e)
        return False


print(exec_answer(answer, 61))

False


## 1.3 Guided Introduction: The Question.

Now we want to automatically generate questions to evaluate the performance of our model. There are benchmarks on this subject, but here we want to practice code by generating the questions ourselves.

We will generate some basic filter questions.

<font color='red'>TODO: Fill in the `generate_filter_question` function following the comments inside.</font>


In [85]:
import random
from tqdm import tqdm


def generate_random_question(generate_function, df, k, seed=42):
    random.seed(seed)
    questions = []
    for _ in tqdm(range(k)):
        question = generate_function(df)
        questions.append(question)
    return questions


def generate_filter_question(df):
    # Get a random target column and a random filter column (be careful they should be different)
    # Get a random filter value inside the filer column. Avoid NaN values.
    target_col = random.choice(df.columns)
    target_val = random.choice(df[target_col].dropna().unique())
    filter_col = random.choice([col for col in df.columns if col != target_col])
    filter_val = random.choice(df[filter_col].dropna().unique())
    # operation = random.choice(numerical_ops)

    # Create a question template that take a target column, a filter column and a filter value
    # WE ARE ALSO ADDING A TARGET VALUE, OTHERWISE THE ANSWER IS IMPOSSIBLE TO COMPUTE
    template = (
        "Generate a question based on this tabular dataset:\n"
        "{df_info}\n"
        "The question should be about the number of rows where the value in the column '{target_col}' is '{target_val}' "
        "and the value in the column '{filter_col}' is '{filter_val}'."
    )

    content = template.format(
        df_info=df.head(5).to_string(index=False),
        target_col=target_col,
        target_val=target_val,
        filter_col=filter_col,
        filter_val=filter_val,
    )

    chat = [
        {"role": "user", "content": content},
    ]
    chat_encoded = tokenizer.apply_chat_template(chat, return_tensors="pt").to(
        llm.device
    )

    llm_outputs_ids = llm.generate(
        input_ids=chat_encoded,
        generation_config=generation_config,
    )[0]
    question = tokenizer.decode(
        llm_outputs_ids[chat_encoded.size(1) :], skip_special_tokens=True
    )

    # Compute the correct answer for the given target column, filter column and filter value.
    correct_answer = df[
        (df[target_col] == target_val) & (df[filter_col] == filter_val)
    ].shape[0]

    # print(df[(df[target_col] == target_val) & (df[filter_col] == filter_val)])
    # print(df[(df[target_col] == target_val) & (df[filter_col] == filter_val)].shape)

    # return formatted question and associated answer in a dict {"question":[question], "answer":[answer]}
    return {"question": question, "answer": correct_answer}


generate_random_question(generate_filter_question, df, k=1)

100%|██████████| 1/1 [00:03<00:00,  3.37s/it]


[{'question': "What is the number of rows where the value in the column 'Embarked' is 'S' and the value in the column 'PassengerId' is '351'?",
  'answer': 1}]

In [86]:
# import random
# import operator


# def generate_random_question(generate_function, df, k=1, seed=42):
#     #random.seed(seed)
#     return [generate_function(df) for _ in range(k)]


# def generate_filter_question(df):
#     categorical_columns = ["Sex", "Pclass", "Embarked", "Survived"]
#     numerical_columns = ["Age", "Fare", "SibSp", "Parch"]
#     numerical_ops = ["<", ">", "==", "!=", "<=", ">="]
#     operators = {
#         "<": operator.lt,
#         ">": operator.gt,
#         "==": operator.eq,
#         "!=": operator.ne,
#         "<=": operator.le,
#         ">=": operator.ge,
#     }

#     # Get a random target column and a random filter column (be careful they should be different)
#     # Get a random filter value inside the filter column. Avoid NaN values.
#     filter_column = random.choice(categorical_columns + numerical_columns)

#     if filter_column in categorical_columns:  # Categorical column
#         filter_value = random.choice(df[filter_column].dropna().unique())
#     else:  # Numerical column
#         filter_ops = random.choice(numerical_ops)
#         filter_value = random.choice(df[filter_column].dropna().unique())

#     # Generate a random target column
#     target_column = random.choice(
#         [
#             col
#             for col in [*categorical_columns, *numerical_columns]
#             if col != filter_column
#         ]
#     )

#     if target_column in categorical_columns:
#         target_value = random.choice(df[target_column].dropna().unique())

#         if target_column == "Survived":
#             target_value = "survived" if target_value == 1 else "not survived"
#         elif target_column == "Pclass":
#             target_value = f"are in class {target_value}"
#         elif target_column == "Embarked":
#             target_value = f"embarked in {target_value}"
#         elif target_column == "Sex":
#             target_value = f"are {target_value}"
#     else:
#         target_ops = random.choice(numerical_ops)
#         target_value = random.choice(df[target_column].dropna().unique())

#     # Create a question template that take a target column, a filter column and a filter value
#     # Compute the correct answer for the given target column, filter column and filter value
#     if filter_column in categorical_columns and target_column in categorical_columns:
#         question = (
#             f"How many passengers {target_value} and {filter_column} is {filter_value}?"
#         )
#         correct_answer = df[
#             (df[filter_column] == filter_value) & (df[target_column] == target_value)
#         ][target_column].count()
#     elif filter_column in numerical_columns and target_column in categorical_columns:
#         question = f"How many passengers {target_value} and {filter_column} {filter_ops} {filter_value}?"
#         correct_answer = df[
#             operators[filter_ops](df[filter_column], filter_value)
#             & (df[target_column] == target_value)
#         ][target_column].count()
#     elif filter_column in categorical_columns and target_column in numerical_columns:
#         question = f"How many passengers {target_column} {target_ops} {target_value} and {filter_column} is {filter_value}?"
#         correct_answer = df[
#             operators[target_ops](df[target_column], target_value)
#             & (df[filter_column] == filter_value)
#         ][target_column].count()
#     else:
#         question = f"How many passengers {target_column} {target_ops} {target_value} and {filter_column} {filter_ops} {filter_value}?"
#         correct_answer = df[
#             operators[target_ops](df[target_column], target_value)
#             & operators[filter_ops](df[filter_column], filter_value)
#         ][target_column].count()

#     # return formatted question and associated answer in a dict {"question":[question], "answer":[answer]}
#     return {"question": question, "answer": correct_answer}


# generate_random_question(generate_filter_question, df, k=5)

## 1.4 Guided Introduction: The Evaluation.

The last step in this section is to evaluate our model on 20 random questions! We'll use simple accuracy.

You should have an accuracy between 0.9 and 1.

<font color='red'>TODO: Follow instruction in comment of the cell below.</font>

<font color='green'>BONUS: Investigate on errors and improve our prompt/parsing to solve them.</font>


In [89]:
# Generate 20 random question
print("Generating random questions")
questions = generate_random_question(generate_filter_question, df, k=20)
sum_correct = 0
incorrect_answers = []

# Iterate over question to format prompt, generate answer and execute answer.
print("\nStarting the evaluation")
for q in tqdm(questions):
    prompt = example_prompt_template.format(
        var_name="df",
        df_info=df.head(5).to_string(index=False),
        user_question=q["question"],
    )
    answer = generate_answer(prompt)
    res = exec_answer(answer, q["answer"])
    if res == False:
        incorrect_answers.append((q, answer))

    sum_correct += res

# Report the Accuracy
print(f"Accuracy: {sum_correct}/{len(questions)}")

Generating random questions


100%|██████████| 20/20 [01:11<00:00,  3.57s/it]



Starting the evaluation


 70%|███████   | 14/20 [01:16<00:31,  5.28s/it]

Output is a DataFrame or Series, please return a scalar.


 90%|█████████ | 18/20 [01:38<00:10,  5.34s/it]

Output is a DataFrame or Series, please return a scalar.


100%|██████████| 20/20 [01:48<00:00,  5.44s/it]

Accuracy: 18/20


In [90]:
print("Incorrect answers:", incorrect_answers)

# WRITE THAT OUR ERRORS ARE DUE TO THE FACT THAT THE OUTPUT IS A DATAFRAME

Incorrect answers: [({'question': "How many rows in the dataset have the value 'Q' in the 'Embarked' column and the value '64.0' in the 'Age' column?", 'answer': 0}, "Python code:\n```python\n# Count the number of rows where 'Embarked' is 'Q' and 'Age' is '64.0'\noutput = (df['Embarked'] == 'Q') & (df['Age'] == 64.0).sum()\nprint(output)\n```"), ({'question': "How many rows in the dataset have the value 'C' in the 'Embarked' column and the value '1' in the 'Pclass' column?", 'answer': 74}, "Python code:\n```python\n# Count the number of rows where 'Embarked' is 'C' and 'Pclass' is '1'\noutput = (df['Embarked'] == 'C') & (df['Pclass'] == 1).sum()\nprint(output)\n```")]


## 2. More Questions.

Now it's your turn to imagine a type of question ("How many ..."). Implement a function to generate new type of question. Verify that our previous code work with your new question then evaluate it.

<font color='red'>TODO: Generate **AT LEAST ONE** new type of question and report this new question accuracy.</font>


In [91]:
import random
from tqdm import tqdm


def generate_fare_question(df):
    # Create a question template that take a target column, a filter column and a filter value
    template = (
        "Generate a question based on this tabular dataset:\n"
        "{df_info}\n"
        "The question should be about the total fare for the passengers where "
        "the value in the column '{filter_col}' is '{filter_val}'."
        "Here are some examples of questions:"
        "1) What is the total fare for the passengers who are in class 3?"
        "2) How much did the passengers who embarked in S pay in total?"
        "3) What is the total fare for male passengers?"
    )

    # Get a random target column and a random filter column (be careful they should be different)
    # Get a random filter value inside the filer column. Avoid NaN values.
    filter_col = random.choice([col for col in df.columns if col != "Fare"])
    filter_val = random.choice(df[filter_col].dropna().unique())

    content = template.format(
        df_info=df.head(5).to_string(index=False),
        filter_col=filter_col,
        filter_val=filter_val,
    )

    chat = [
        {"role": "user", "content": content},
    ]
    chat_encoded = tokenizer.apply_chat_template(chat, return_tensors="pt").to(
        llm.device
    )

    llm_outputs_ids = llm.generate(
        input_ids=chat_encoded,
        generation_config=generation_config,
    )[0]
    question = tokenizer.decode(
        llm_outputs_ids[chat_encoded.size(1) :], skip_special_tokens=True
    )

    # Compute the correct answer for the given target column, filter column and filter value.
    correct_answer = df[df[filter_col] == filter_val]["Fare"].sum()

    # return formatted question and associated answer in a dict {"question":[question], "answer":[answer]}
    return {"question": question, "answer": correct_answer}

In [92]:
# Generate 20 random question
print("Generating random questions")
questions = generate_random_question(generate_fare_question, df, k=20)
sum_correct = 0
incorrect_answers = []

# Iterate over question to format prompt, generate answer and execute answer.
print("\nStarting the evaluation")
for q in tqdm(questions):
    prompt = example_prompt_template.format(
        var_name="df",
        df_info=df.head(5).to_string(index=False),
        user_question=q["question"],
    )
    answer = generate_answer(prompt)
    res = exec_answer(answer, q["answer"])
    if res == False:
        incorrect_answers.append((prompt, answer))

    sum_correct += res

# Report the Accuracy
print(f"Accuracy: {sum_correct}/{len(questions)}")

Generating random questions


  5%|▌         | 1/20 [00:02<00:48,  2.53s/it]

What is the total fare for the passengers where the value in the column 'Survived' is '0'?


 10%|█         | 2/20 [00:04<00:37,  2.10s/it]

What is the total fare paid by male passengers?


...

What is the total fare for the passengers where the value in the column 'SibSp' is '1'?


100%|██████████| 20/20 [00:51<00:00,  2.56s/it]


What is the total fare for the passengers where the value in the column 'Age' is '71.0'?

Starting the evaluation


100%|██████████| 20/20 [01:55<00:00,  5.78s/it]

Accuracy: 20/20


## 3. More datasets.

Below we load a new dataset: "adult_income_dataset".

<font color='red'>TODO: Evaluate our questions on this new dataset. Report the accuracy. Comment Any differences.</font>

<font color='green'>BONUS: Try to find a prompt that answer this question: What is the mean salary of titanic surviror based on adult dataset.</font>


In [94]:
adult = pd.read_csv("hf://datasets/meghana/adult_income_dataset/adult.csv")
adult.info()

titanic = df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB
